In [1]:
import json
from utils import QAFileConvertor
from Dataset import QAContainer
from transformers import BertTokenizer

In [2]:
# with open("Data/socialiqa-train-dev/dev.jsonl", "r") as json_file:
#     json_list = list(json_file)

# for json_str in json_list:
#     result = json.loads(json_str)
#     print(f"result: {result}")
a = QAFileConvertor("Data/socialiqa-train-dev/train.jsonl", "Data/socialiqa-train-dev/train-labels.lst")

In [3]:
a.convert("Data/socialiqa-train-dev/train_converted.jsonl")

In [33]:
b = QAContainer("Data/socialiqa-train-dev/train_converted.jsonl")

In [10]:
tokenizer =  BertTokenizer.from_pretrained("bert-base-uncased",truncation_side="left")

c:\Users\yunha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yunha\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [34]:
text=1
while text!=0:
    text = next(b,0)
    if text!=0:
        context = text['context']
        question = text['question']
        answer = text['answer']
        tokenized_context = tokenizer(context,padding="max_length", truncation=True,max_length=16)
        tokenized_question = tokenizer(question, padding = 'max_length',truncation = True, max_length = 16)
        tokenized_answer = tokenizer(answer, padding='max_length', truncation=True, max_length = 16)


In [23]:
tokenizer(test_text['context'],padding="max_length", truncation=True,max_length=16)

{'input_ids': [101, 7232, 2787, 2000, 2031, 1037, 26375, 1998, 5935, 2014, 2814, 2362, 1012, 102, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]}

In [31]:
tokenized_context

{'input_ids': [101, 8804, 2001, 11654, 2011, 2032, 2061, 2016, 2741, 2032, 2185, 1012, 102, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}